In [3]:
import pandas as pd
import spacy

# Load the parquet file to analyze its content
file_path = r"C:\Users\charl\Documents\ai-plz-lmao\data\datasets\part-00000-77745408-e261-4890-8d8b-0832eb0a7130-c000.snappy.parquet"
df = pd.read_parquet(file_path)

# Display basic information about the file
df_info = df.info()
df_head = df.head()

df['date'].dtype

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2399 entries, 0 to 2398
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   content       2399 non-null   object
 1   content_type  2399 non-null   object
 2   date          2399 non-null   object
 3   url           2399 non-null   object
dtypes: object(4)
memory usage: 75.1+ KB


dtype('O')

In [4]:
df[df['url'].str.contains('nasdaq') == True]

,content,content_type,date,url


In [5]:
from bs4 import BeautifulSoup

sample = df.iloc[22]['content'].decode('utf-8')
# Step 2: Parse the HTML string using BeautifulSoup
soup = BeautifulSoup(sample, 'html.parser')

# Example: Extract the title of the HTML page
title = soup.title.string
print("Page Title:", title)

Page Title: 屏東滿洲鄉~港口吊橋,樹屋,港仔大沙漠 - 沙灘6.jpg－Annys life｜痞客邦 PIXNET


In [6]:
chunked_content = [para.get_text() for para in soup.find_all("p")]

In [8]:
!python -m spacy download en_core_web_sm
nlp = spacy.load("en_core_web_sm")


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     --------------------------------------- 0.0/12.8 MB 330.3 kB/s eta 0:00:39
     --------------------------------------- 0.1/12.8 MB 919.0 kB/s eta 0:00:14
     ----- ---------------------------------- 1.7/12.8 MB 13.5 MB/s eta 0:00:01
     ---------- ----------------------------- 3.4/12.8 MB 19.9 MB/s eta 0:00:01
     ----------------- ---------------------- 5.6/12.8 MB 25.7 MB/s eta 0:00:01
     ------------------------ --------------- 7.7/12.8 MB 29.1 MB/s eta 0:00:01
     ------------------------------ --------- 9.7/12.8 MB 31.0 MB/s eta 0:00:01
     ------------------------------------ -- 11.8/12.8 MB 43.5 MB/s eta 0:00:01
     --------------------------------------  12.7/12.8 MB 46.7 MB/s eta 0:00:01
     --------------------------------------  12.8/12.8 MB 40.9 MB/s eta 0:00:01
     --------------------------------------- 12.8/12.8 MB 32.7 MB/s eta 0:00:00
✔ Download and installation successful
You can n

In [34]:
# Tokenize content
doc = nlp("Google is looking at buying U.K. startup for $1 billion")

# Get NeR
# if ent.label_ == [ORG], save special
for ent in doc.ents:
    if ent.label_ == 'ORG':
        print(type(ent.text))

<class 'str'>


<h5> Potential feature -- add NeR labels for all Fortune 500 companies</h5>

In [28]:
from spacy.tokens import Span
nlp = spacy.load("en_core_web_sm")
doc = nlp("fb is hiring a new vice president of global policy")
ents = [(e.text, e.start_char, e.end_char, e.label_) for e in doc.ents]
print('Before', ents)
# The model didn't recognize "fb" as an entity :(

# Create a span for the new entity
fb_ent = Span(doc, 0, 1, label="ORG")
orig_ents = list(doc.ents)

# Option 1: Modify the provided entity spans, leaving the rest unmodified
doc.set_ents([fb_ent], default="unmodified")

# Option 2: Assign a complete list of ents to doc.ents
doc.ents = orig_ents + [fb_ent]

ents = [(e.text, e.start, e.end, e.label_) for e in doc.ents]
print('After', ents)
# [('fb', 0, 1, 'ORG')] 🎉

Before []
After [('fb', 0, 1, 'ORG')]
